## Exercise 2: Diagnosing what and why

This notebook demonstrates how to hunt and diagnose datasets for malicious activity. This dataset is something that a Threat Hunter, Security Operations (SOC) analyst or a detection engineer will encounter in their day-to-day role. We'll use a large language model library called **PandasAI** that provides an LLM interface to dataframes to explore, query and diagnose issues

**What's the story?**

You are a threat hunter who is proactively looking to secure your organization. You create a hypothesis that you will find some sneaky malicious activity and start looking at network data. This is your exploratory data analysis (EDA) process.


### Key Questions:
- What can I explore in the data that I have? 
- Is there a strong dependency of certain fields on one another? How do these interactions play out?

# Imports

In [7]:
import pandas as pd
import pandasai as pai 
import os

from dotenv import load_dotenv

# pandasai imports
from pandasai.llm.openai import OpenAI
from pandasai import SmartDataframe
from pandasai import clear_cache
from pandasai import Agent

# Initialization

In [2]:
READ_FROM_PICKLE = True

In [ ]:
# finds .env file and loads the vars
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY", "Key not found")

In [5]:
if READ_FROM_PICKLE:
    malicious_df = pd.read_pickle("../data/malicious_features_numeric.pkl")
    benign_df = pd.read_pickle("../data/benign_features_numeric.pkl")

In [8]:
# Instantiate a LLM
llm = OpenAI(api_token=openai_api_key)

In [9]:
malicious_smart = SmartDataframe(
    df=pd.DataFrame(malicious_df),
    config={"llm": llm, "verbose": True},
    name="Mirai botnet network packet capture.",
    description="A dataframe that is derived from a packet capture of the Mirai botnet network traffic.",
)

In [10]:
benign_smart = SmartDataframe(
    df=pd.DataFrame(benign_df),
    config={"llm": llm, "verbose": True},
    name="Packet capture of normal operation network travvid.",
    description="A dataframe that is derived from a packet capture of the regular operation of a network.",
)

In [ ]:
malicious_smart.columns

# EDA

## Statistical

### Exploration via prompting

In [11]:
top_5_source_IPs = malicious_smart.chat("Which are the 5 most popular source IP addresses?")

top_5_source_IPs

2025-04-04 04:59:09 [INFO] Question: Which are the 5 most popular source IP addresses?
2025-04-04 04:59:09 [INFO] Running PandasAI with openai LLM...
2025-04-04 04:59:09 [INFO] Prompt ID: 66d7af6d-7b52-4269-9da3-63e27dbc4f4e
2025-04-04 04:59:09 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 04:59:09 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 04:59:09 [INFO] Executing Step 1: CacheLookup
2025-04-04 04:59:09 [INFO] Executing Step 2: PromptGeneration
2025-04-04 04:59:09 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    - name: Payload
      type: o

Traceback (most recent call last):
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 283, in run
    output = (self.code_generation_pipeline | self.code_execution_pipeline).run(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 136, in execute
    {"content_type": "response", "value": ResponseSerializer.serialize(result)},
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/responses/response_serializer.py", line 25, in serialize
    df_dict = ResponseS

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'columns'\n"

In [12]:
print(malicious_smart.last_code_generated)

df = dfs[0]
top_source_ips = df['Numeric Source IP'].value_counts().head(5)
result = {'type': 'dataframe', 'value': top_source_ips}


In [13]:
top_5_dst_ports = malicious_smart.chat("Find the most used destination ports.")
top_5_dst_ports

2025-04-04 04:59:45 [INFO] Question: Find the most used destination ports.
2025-04-04 04:59:45 [INFO] Running PandasAI with openai LLM...
2025-04-04 04:59:45 [INFO] Prompt ID: fefa842a-2f90-48b3-bf38-e2094c123e12
2025-04-04 04:59:45 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 04:59:45 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 04:59:45 [INFO] Executing Step 1: CacheLookup
2025-04-04 04:59:45 [INFO] Executing Step 2: PromptGeneration
2025-04-04 04:59:45 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    - name: Payload
      type: object
      

,Destination Port,Count
0,10240.0,62584
1,53.0,33348
2,80.0,17644
3,23.0,9466
4,21897.0,7158


In [ ]:
print(malicious_smart.last_code_generated)

### Prompt engineering

In [14]:
rare_ports = malicious_smart.chat(
    "Which are the most commonly used known destination ports?"
)
rare_ports

2025-04-04 05:05:06 [INFO] Question: Which are the most commonly used known destination ports?
2025-04-04 05:05:06 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:05:06 [INFO] Prompt ID: b839e675-4293-4e4b-bfee-fe7891f82da2
2025-04-04 05:05:06 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:05:06 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:05:06 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:05:06 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:05:06 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    - name: Payload
     

Traceback (most recent call last):
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 283, in run
    output = (self.code_generation_pipeline | self.code_execution_pipeline).run(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 136, in execute
    {"content_type": "response", "value": ResponseSerializer.serialize(result)},
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/responses/response_serializer.py", line 25, in serialize
    df_dict = ResponseS

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'columns'\n"

In [15]:
rare_ports = malicious_smart.chat(
    "Which are the most used destination ports less than or equal to 1024?"
)
rare_ports

2025-04-04 05:05:30 [INFO] Question: Which are the most used destination ports less than or equal to 1024?
2025-04-04 05:05:30 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:05:30 [INFO] Prompt ID: 9a0b31bc-92c3-4bdc-84a7-4f6d4fbbbce3
2025-04-04 05:05:30 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:05:30 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:05:30 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:05:30 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:05:30 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    - name: P

Traceback (most recent call last):
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 283, in run
    output = (self.code_generation_pipeline | self.code_execution_pipeline).run(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 136, in execute
    {"content_type": "response", "value": ResponseSerializer.serialize(result)},
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/responses/response_serializer.py", line 25, in serialize
    df_dict = ResponseS

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'columns'\n"

### Correlation

In [16]:
correlation = malicious_smart.chat(
    """1. Calculate the correlation between the source port and the length of a packet.
       2. Explain if the correlation that you calculated is significant and why.
    """
)
correlation

2025-04-04 05:05:53 [INFO] Question: 1. Calculate the correlation between the source port and the length of a packet.
       2. Explain if the correlation that you calculated is significant and why.
    
2025-04-04 05:05:53 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:05:53 [INFO] Prompt ID: a4188ce8-add0-426e-8c6d-ecfd3329d830
2025-04-04 05:05:53 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:05:53 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:05:54 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:05:54 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:05:54 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination P

'The correlation between the source port and the length of a packet is -0.16566431740345766. The correlation is not significant.'

### Hypothesis testing
- Is there sufficient reason to believe that one data points has interaction with another? 
- If yes, how do we quantify it?

In [ ]:
hypothesis = malicious_smart.chat(
    "Is the difference between dst_ip_total_bytes and Packet Length statistically significant?"
)
hypothesis

2025-04-04 05:06:48 [INFO] Question: Is the difference between dst_ip_total_bytes and Packet Length statistically significant?
2025-04-04 05:06:48 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:06:48 [INFO] Prompt ID: 06770268-bf46-4c50-a21c-0e58e38eaed4
2025-04-04 05:06:48 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:06:48 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:06:48 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:06:48 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:06:48 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 

'The difference between dst_ip_total_bytes and Packet Length is statistically significant.'

### Outliers

In [18]:
outliers = malicious_smart.chat(
    "Find the z score of the Packet Length and then calculate the top ten outliers."
)
outliers

2025-04-04 05:07:08 [INFO] Question: Find the z score of the Packet Length and then calculate the top ten outliers.
2025-04-04 05:07:08 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:07:08 [INFO] Prompt ID: e4ccf5be-0806-4eb5-a593-2ac61ffa20ed
2025-04-04 05:07:08 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:07:08 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:07:08 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:07:08 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:07:08 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    

,Timestamp,Source Port,Destination Port,Payload,Packet Length,Protocol,src_ip_total_bytes,dst_ip_total_bytes,Numeric Source IP,Numeric Destination IP,dst_port_freq_encoded,Interarrival,Packet Length Z Score
581,1.540446e+09,1215.0,1900.0,Raw,166,17.0,2000,311,3232236147,4026531834,0.006827,0.107952,6.874131
582,1.540446e+09,1215.0,1900.0,Raw,166,17.0,2166,477,3232236147,4026531834,0.006827,0.000148,6.874131
583,1.540446e+09,1215.0,1900.0,Raw,166,17.0,2332,643,3232236147,4026531834,0.006827,0.000002,6.874131
1012,1.540446e+09,67.0,68.0,BOOTP,166,17.0,1791,166,3232236033,4294967295,0.000883,0.034520,6.874131
1351,1.540446e+09,1215.0,1900.0,Raw,166,17.0,5709,809,3232236147,4026531834,0.006827,0.083352,6.874131
1352,1.540446e+09,1215.0,1900.0,Raw,166,17.0,5875,975,3232236147,4026531834,0.006827,0.007626,6.874131
1353,1.540446e+09,1215.0,1900.0,Raw,166,17.0,6041,1141,3232236147,4026531834,0.006827,0.000676,6.874131
2257,1.540446e+09,1215.0,1900.0,Raw,166,17.0,9468,1597,3232236147,4026531834,0.006827,0.065097,6.874131
2258,1.540446e+09,1215.0,1900.0,Raw,166,17.0,9634,1763,3232236147,4026531834,0.006827,0.001044,6.874131
2259,1.540446e+09,1215.0,1900.0,Raw,166,17.0,9800,1929,3232236147,4026531834,0.006827,0.001050,6.874131


## Visualizations

In [25]:
malicious_smart.chat(
    "Plot the heatmap of the correlations of all variables."
)

2025-04-04 05:10:33 [INFO] Question: Plot the heatmap of the correlations of all variables.
2025-04-04 05:10:33 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:10:33 [INFO] Prompt ID: 0cf31de3-0322-4d65-b715-4ddfd342c74c
2025-04-04 05:10:33 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:10:33 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:10:33 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:10:33 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:10:33 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    - name: Payload
      ty

Traceback (most recent call last):
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 283, in run
    output = (self.code_generation_pipeline | self.code_execution_pipeline).run(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 116, in execute
    raise e
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 87, in execute
    result = self.execute_code(input, code_context)
  File "/Users/preeti/preeti_gh/ai_for_sec

"Unfortunately, I was not able to answer your question, because of the following error:\n\ncould not convert string to float: 'Padding'\n"

In [26]:
print(malicious_smart.last_code_executed)

df = pd.concat(dfs)
correlation_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap of Correlations of All Variables')
plt.show()
result = {'type': 'plot', 'value': '/Users/preeti/preeti_gh/ai_for_sec_workshop/exports/charts/temp_chart.png'}


In [ ]:
malicious_smart.chat("Plot a histogram for each of the variables.")

In [ ]:
print(malicious_smart.last_code_executed)

# Agents

In [19]:
from pandasai import Agent

In [20]:
agent = Agent(malicious_smart, config={"llm": llm},memory_size=1000)

In [21]:
agent.chat('Is this dataframe indicating malicious or benign network behavior?')

2025-04-04 05:07:55 [INFO] Question: Is this dataframe indicating malicious or benign network behavior?
2025-04-04 05:07:55 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:07:55 [INFO] Prompt ID: 199ad1b5-a2a6-4c71-8280-5ebeec37f6eb
2025-04-04 05:07:55 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:07:55 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:07:55 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:07:55 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:07:55 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
      samples:
      - '2008687209'
      - '7024883495'
      - '3649210408'
    - name: Source Port
      type: float64
      samples:
      - 59268.0
      - 34340.0
      - 7935.0
    - name: Destination Port
      type: float64
      samples:
      - 3947.0
      - 4234.0
      - 2518.0
    - name: Payl

Traceback (most recent call last):
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 283, in run
    output = (self.code_generation_pipeline | self.code_execution_pipeline).run(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_generator.py", line 33, in execute
    code = pipeline_context.config.llm.generate_code(input, pipeline_context)
  File "/Users/preeti/preeti_gh/ai_for_sec_workshop/.venv/lib/python3.10/site-packages/pandasai/llm/base.py", line 197, in generate_code
    return self._extract_code(response)
  F

'Unfortunately, I was not able to answer your question, because of the following error:\n\nNo code found in the response\n'

In [22]:
agent.chat('The dataframe that you have been given is a packet capture of computer network traffic. It has numerical features that characterize the packets that have been observed in this network. A packet capture is malicious if it has too many requests to ports that are unusual. Usual ports are 23 for Telnet, 22 for SSH and 80 for HTTP. Is this packet capture malicious or benign?')

2025-04-04 05:08:42 [INFO] Question: The dataframe that you have been given is a packet capture of computer network traffic. It has numerical features that characterize the packets that have been observed in this network. A packet capture is malicious if it has too many requests to ports that are unusual. Usual ports are 23 for Telnet, 22 for SSH and 80 for HTTP. Is this packet capture malicious or benign?
2025-04-04 05:08:42 [INFO] Running PandasAI with openai LLM...
2025-04-04 05:08:42 [INFO] Prompt ID: a7542adc-650d-4a0f-b1d1-f8942516c30b
2025-04-04 05:08:43 [INFO] Executing Pipeline: GenerateChatPipeline
2025-04-04 05:08:43 [INFO] Executing Step 0: ValidatePipelineInput
2025-04-04 05:08:43 [INFO] Executing Step 1: CacheLookup
2025-04-04 05:08:43 [INFO] Executing Step 2: PromptGeneration
2025-04-04 05:08:43 [INFO] Using prompt: dfs[0]:
  name: null
  description: null
  type: pd.DataFrame
  rows: 154090
  columns: 12
  schema:
    fields:
    - name: Timestamp
      type: object
   

'Malicious network behavior detected'

In [23]:
agent.explain()

2025-04-04 05:08:54 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-04 05:08:54 [INFO] Explanation:  I looked at the data that shows where the network traffic is going and coming from. I checked if the destination ports were unusual, meaning they were not common ports like 23, 22, or 80. If there were a lot of packets going to unusual ports, then I concluded that it might be malicious behavior. Otherwise, if most of the traffic was going to common ports, then it was likely benign behavior.
                


'I looked at the data that shows where the network traffic is going and coming from. I checked if the destination ports were unusual, meaning they were not common ports like 23, 22, or 80. If there were a lot of packets going to unusual ports, then I concluded that it might be malicious behavior. Otherwise, if most of the traffic was going to common ports, then it was likely benign behavior.'

# Clear cache

In [24]:
clear_cache()